In [2]:
import pandas as pd

In [3]:
from google.colab import files

# Dosya yükleme
uploaded = files.upload()

Saving e-ticaret_urun_yorumlari.csv to e-ticaret_urun_yorumlari.csv


In [4]:
df = pd.read_csv('e-ticaret_urun_yorumlari.csv', sep=';', on_bad_lines='skip')
df.head()

,Metin,Durum
0,evet anlatıldığı gibi,1
1,Daha öncede almıştım bu cihazdan ense ve sakal...,1
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,1
3,Daha öncede aynısını almıştım çok güzel ve kal...,1
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1


In [5]:
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer

In [6]:
# Etiketleri sayısal değerlere dönüştürme (0: Olumsuz, 1: Olumlu, 2: Nötr)
label_encoder = LabelEncoder()
df['Durum'] = label_encoder.fit_transform(df['Durum'])

In [7]:
# Tokenizer'ı yükleyelim
model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

In [8]:
# Tokenization fonksiyonu
def tokenize_function(examples):
    return tokenizer(examples["Metin"], padding="max_length", truncation=True)

# Tokenize etme
tokenized_data = df['Metin'].apply(lambda x: tokenizer(x, padding="max_length", truncation=True, max_length=128))

# Tokenize edilmiş veriyi kontrol et
tokenized_data.head()

,Metin
0,"[input_ids, token_type_ids, attention_mask]"
1,"[input_ids, token_type_ids, attention_mask]"
2,"[input_ids, token_type_ids, attention_mask]"
3,"[input_ids, token_type_ids, attention_mask]"
4,"[input_ids, token_type_ids, attention_mask]"


In [10]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [11]:
from datasets import Dataset

# Tokenize edilmiş veriyi Dataset formatına dönüştürme
dataset = Dataset.from_dict({
    'input_ids': tokenized_data.apply(lambda x: x['input_ids']).tolist(),
    'attention_mask': tokenized_data.apply(lambda x: x['attention_mask']).tolist(),
    'labels': df['Durum'].tolist()  # Etiketler (0: Olumsuz, 1: Olumlu, 2: Nötr)
})

In [12]:
from transformers import AutoModelForSequenceClassification

# Modeli yükleyelim
model_name = "dbmdz/bert-base-turkish-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 etiket (Olumsuz, Olumlu, Nötr)


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import os
os.environ["WANDB_DISABLED"] = "true"
from sklearn.metrics import accuracy_score
# Tokenization fonksiyonu - dataset içinde kullanılacak
def tokenize_function(examples):
    return tokenizer(examples["Metin"], padding="max_length", truncation=True, max_length=128)

# DataFrame'den Hugging Face Dataset'e dönüşüm
dataset = Dataset.from_pandas(df)

# Dataset'i tokenleştirme
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Modeli yükleyelim
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 etiket
small_train_dataset = tokenized_datasets.shuffle(seed=42).select([i for i in list(range(0, len(tokenized_datasets)//5))])

# Eğitim parametreleri
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# compute_metrics fonksiyonunu tanımlayalım
def compute_metrics(p):
    predictions, labels = p
    preds = predictions.argmax(axis=-1)  # Modelin tahmin ettiği etiketler
    acc = accuracy_score(labels, preds)  # Doğruluk oranını hesapla
    return {"accuracy": acc}

# Trainer parametrelerini ekleyelim
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset.rename_column("Durum", "labels"),  # "Durum" sütununu "labels" olarak yeniden adlandır
    eval_dataset=small_train_dataset.rename_column("Durum", "labels"),  # Test verisi için de aynı değişikliği yapın
    compute_metrics=compute_metrics  # compute_metrics fonksiyonu ekliyoruz
)

# Modeli tekrar eğitmek için:
trainer.train()
# Eğitim tamamlandıktan sonra doğruluk değerini hesaplayabiliriz
eval_results = trainer.evaluate()  # Eval dataset üzerinde değerlendirme yapıyoruz
print(f"Evaluation Results: {eval_results}")

Map:   0%|          | 0/15170 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.342200,0.183624,0.938365


Epoch,Training Loss,Validation Loss,Accuracy
1,0.342200,0.183624,0.938365
2,0.186500,0.109549,0.966051
3,0.105900,0.094623,0.974951


Evaluation Results: {'eval_loss': 0.09462297707796097, 'eval_accuracy': 0.974950560316414, 'eval_runtime': 1304.7356, 'eval_samples_per_second': 2.325, 'eval_steps_per_second': 0.146, 'epoch': 3.0}


In [14]:
# Modeli kaydetme
model.save_pretrained("./duygu_analizi_modeli")
tokenizer.save_pretrained("./duygu_analizi_modeli")

('./duygu_analizi_modeli/tokenizer_config.json',
 './duygu_analizi_modeli/special_tokens_map.json',
 './duygu_analizi_modeli/vocab.txt',
 './duygu_analizi_modeli/added_tokens.json',
 './duygu_analizi_modeli/tokenizer.json')

In [18]:
label_mapping = {
    0: "Olumsuz",
    1: "Olumlu",
    2: "Nötr"
}

In [19]:
# Yeni bir veri örneği üzerinden tahmin yapma
sample_text = ["Bu ürün çok güzel, memnun kaldım!"]
inputs = tokenizer(sample_text, return_tensors="pt", padding=True, truncation=True, max_length=128)
outputs = model(**inputs)

# Tahmin edilen sınıf
predictions = outputs.logits.argmax(dim=-1)
predicted_label = label_mapping[predictions.item()]  # Sayısal etiketi anlamlı bir etikete dönüştür

print(f"Tahmin Edilen Durum: {predicted_label}")

Tahmin Edilen Durum: Olumlu


In [20]:
from google.colab import files
import shutil

# Model ve tokenizer dosyalarını bir zip dosyası olarak sıkıştır
shutil.make_archive('duygu_analizi_modeli', 'zip', './duygu_analizi_modeli')

# Zip dosyasını indir
files.download('duygu_analizi_modeli.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>